In [17]:
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored
from sklearn.model_selection import train_test_split
from sksurv.util import Surv
import pandas as pd
import numpy as np

In [18]:
dataset = pd.read_csv("data_ready_final.csv")

C:\Users\Omar\AppData\Local\Temp\ipykernel_3036\2246386907.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("data_ready_wnw.csv")


In [5]:
# dataset = dataset[dataset['time_frame'] != 30]
# dataset = dataset[dataset['time_frame'] != 365]
# dataset = dataset[dataset['time_frame'] != 365*5]
# dataset = dataset[dataset['time_frame'] != 365*10]

In [20]:
data_kp = dataset.copy()

In [21]:
data_cox = dataset.copy()

In [22]:
y = Surv.from_dataframe('GRF_STAT_PA_didFail', 'time_frame', dataset)
dataset = dataset.drop(columns=['GRF_STAT_PA_didFail', 'time_frame', "DIAB"])
X = dataset

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
rsf = RandomSurvivalForest(n_estimators=150, max_depth = 10, n_jobs=-1, random_state=42)

In [25]:
rsf.fit(X_train, y_train)

RandomSurvivalForest(max_depth=10, n_estimators=150, n_jobs=-1, random_state=42)

In [56]:
import optuna
import numpy as np
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import cumulative_dynamic_auc
from sklearn.model_selection import train_test_split
from sksurv.util import Surv

# Example dataset
# Replace event_train_col, time_train_col, event_test_col, and time_test_col with your actual data.
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)


# Create structured array for survival data (y_train and y_test)
y_train = Surv.from_arrays(event=y_train["GRF_STAT_PA_didFail"], time=y_train["time_frame"])
y_test = Surv.from_arrays(event=y_test["GRF_STAT_PA_didFail"], time=y_test["time_frame"])

# Define the objective function for Optuna optimization
def objective(trial):
    # Suggest hyperparameters for RSF
    n_estimators = trial.suggest_int('n_estimators', 50, 150)
    max_depth = trial.suggest_int('max_depth', 5, 15)
    min_samples_split = trial.suggest_int('min_samples_split', 5, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 15)
    
    # Define the RSF model with suggested hyperparameters
    rsf = RandomSurvivalForest(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
        n_jobs=2
    )
    
    # Fit the model on the training set
    rsf.fit(X_train, y_train)

    # Predict risk scores on the validation set
    risk_scores = rsf.predict(X_test)

    # Compute time-dependent AUC using the cumulative dynamic AUC function
    time_points = [30, 365, 365*3, 365*5, 365*10]  # Define appropriate time points
    auc, mean_auc = cumulative_dynamic_auc(y_train, y_test, risk_scores, time_points)
    
    # Use mean AUC as the evaluation metric
    if trial.should_prune():
        raise optuna.TrialPruned()

    # Return the negative mean AUC (since Optuna minimizes by default, we negate it to maximize AUC)
    return -mean_auc

# Set up a pruner to stop unpromising trials early
pruner = optuna.pruners.MedianPruner()

# Create an Optuna study
study = optuna.create_study(direction='minimize', pruner=pruner)  # We're minimizing -AUC to maximize AUC

# Optimize the study
study.optimize(objective, n_trials=50)  # Run 50 trials

# Print the best hyperparameters
print("Best Hyperparameters:", study.best_params)

# Print the best score
print("Best AUC:", -study.best_value)  # Revert back to positive AUC


In [45]:
import numpy as np
import pandas as pd
from sksurv.metrics import concordance_index_censored
from sklearn.utils import shuffle

# Define a custom permutation importance function
def permutation_importance_survival(model, X, y, n_repeats=5, random_state=42):
    rng = np.random.RandomState(random_state)
    
    # Compute baseline score (C-index) on original data
    y_time, y_event = y['time_frame'], y['GRF_STAT_PA_didFail']
    baseline_cindex = concordance_index_censored(y_event, y_time, model.predict(X))[0]
    
    feature_importances = {}
    
    # Iterate over each feature in the dataset
    for col in X.columns:
        scores = []
        
        # Shuffle the feature n_repeats times and compute C-index
        for _ in range(n_repeats):
            X_permuted = X.copy()
            X_permuted[col] = shuffle(X[col], random_state=rng)
            permuted_cindex = concordance_index_censored(y_event, y_time, model.predict(X_permuted))[0]
            scores.append(permuted_cindex)
        
        # The importance score is the reduction in C-index after shuffling the feature
        feature_importances[col] = baseline_cindex - np.mean(scores)
    
    # Convert the results to a DataFrame for easier analysis
    importance_df = pd.DataFrame({
        "importances_mean": list(feature_importances.values()),
        "importances_std": np.std([list(feature_importances.values()) for _ in range(n_repeats)], axis=0),
    }, index=X.columns).sort_values(by="importances_mean", ascending=False)
    
    return importance_df

# Assuming you already have the model `rsf` trained and X_test, y_test are defined
# Example usage:
importances = permutation_importance_survival(rsf, X_test, y_test, n_repeats=5)
print(importances)


In [26]:
risk_scores_test = rsf.predict(X_test)


In [27]:

c_index = concordance_index_censored(y_test['GRF_STAT_PA_didFail'], y_test['time_frame'], risk_scores_test)
print(f"Test Concordance Index: {c_index}")

Test Concordance Index: (0.6326084333235508, 5169022, 3001944, 0, 3318)


In [9]:
import matplotlib.pyplot as plt
from sksurv.nonparametric import kaplan_meier_estimator

# Assume y_train contains the structured array with 'event' and 'duration'
# We calculate the censoring survival curve, so we use ~y_train['event']
time, prob_censoring = kaplan_meier_estimator(~y_train['GRF_STAT_PA_didFail'], y_train['time_frame'])

# Plot the Kaplan-Meier curve for censoring
plt.step(time, prob_censoring, where="post")
plt.xlabel("Time")
plt.ylabel("Probability of Not Being Censored")
plt.title("Kaplan-Meier Curve for Censoring Survival Function")
plt.grid(True)
plt.show()


In [10]:
from sksurv.nonparametric import kaplan_meier_estimator
import numpy as np

# Assume y_train contains the structured array with 'event' and 'duration'
time, prob_censoring = kaplan_meier_estimator(~y_train['GRF_STAT_PA_didFail'], y_train['time_frame'])

# Find the maximum time point where the censoring survival function is greater than zero
valid_times = time[prob_censoring > 0]

# The last valid time point
max_valid_time = valid_times[-1] -1
print(f"Maximum valid time point where censoring is > 0: {max_valid_time}")


In [ ]:
import pandas as pd
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.model_selection import train_test_split

# Assuming 'df' is your full dataset containing 'duration', 'event_occurred', and covariates

# Split the dataset into training and test sets
df_train, df_test = train_test_split(data_cox, test_size=0.2, random_state=42)

# Fit the Cox Proportional Hazards model on the training data
cox_model = CoxPHFitter(penalizer=0.1).fit(df_train, duration_col='time_frame', event_col='GRF_STAT_PA_didFail')

# Print the summary of the model
# cox_model.print_summary()
hazard_ratios = cox_model.hazard_ratios_

# Sort by absolute importance
sorted_hazard_ratios = hazard_ratios.sort_values(ascending=False)[:10]

print("Sorted Hazard Ratios:")
print(sorted_hazard_ratios)
# Get the C-Index on the training set
c_index_train = cox_model.concordance_index_
print(f"Concordance Index (Training Set): {c_index_train}")

# Predict risk scores for the test data
test_predictions = cox_model.predict_partial_hazard(df_test)

# Calculate the Concordance Index on the test data
c_index_test = concordance_index(df_test['time_frame'], -test_predictions, df_test['GRF_STAT_PA_didFail'])
print(f"Concordance Index (Test Set): {c_index_test}")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute the correlation matrix
corr_matrix = data_cox.corr()

# Plot the heatmap to visualize correlation
import pandas as pd

# Assume df is your DataFrame with 113 features
# Compute the correlation matrix

# Unstack the correlation matrix to get pairs of features and their correlation
corr_pairs = corr_matrix.unstack()

# Convert to DataFrame for easier filtering and sorting
corr_pairs = pd.DataFrame(corr_pairs, columns=['correlation']).reset_index()

# Rename columns for clarity
corr_pairs.columns = ['Feature1', 'Feature2', 'Correlation']

# Remove self-correlations (where Feature1 == Feature2)
corr_pairs = corr_pairs[corr_pairs['Feature1'] != corr_pairs['Feature2']]

# Sort by absolute correlation value (highest first)
corr_pairs['abs_corr'] = corr_pairs['Correlation'].abs()
sorted_corr_pairs = corr_pairs.sort_values(by='abs_corr', ascending=False)

# Drop the auxiliary abs_corr column
sorted_corr_pairs.drop(columns=['abs_corr'], inplace=True)

# Show the top correlations
print(sorted_corr_pairs.head(20))  # Change the number to show more or fewer correlations


In [ ]:
from sksurv.metrics import integrated_brier_score, brier_score
import numpy as np

# Define the time points at which to compute the Brier score
times = [30, 365, 365*5, 365*10]


# Predict survival functions for the test set
surv_funcs = rsf.predict_survival_function(X_test)

# Convert survival functions into probabilities at specific time points
preds = np.asarray([[fn(t) for t in times] for fn in surv_funcs])

# Compute Brier Score at specific times
brier_scores = brier_score(y_train, y_test, preds, times)
print(f"Brier Scores: {brier_scores}")

# Compute the Integrated Brier Score (IBS) over the specified time points
ibs = integrated_brier_score(y_train, y_test, preds, times)
print(f"Integrated Brier Score (IBS): {ibs}")


In [ ]:
surv_funcs = cox_model.predict_survival_function(df_test)

# Define the time points at which to compute the Brier score (30 days, 1 year, 5 years, 10 years)
times = [30, 365, 365 * 5, 365 * 10]

# Extract survival probabilities at the specific time points
preds = np.asarray([surv_funcs.loc[t].values for t in times]).T  # Transpose to match (n_samples, n_times)

# Convert the test data into the required format for sksurv
y_test = np.array([(status == 1, time) for status, time in zip(df_test['GRF_STAT_PA_didFail'], df_test['time_frame'])],
                  dtype=[('event', '?'), ('time', '<f8')])

y_train = np.array([(status == 1, time) for status, time in zip(df_train['GRF_STAT_PA_didFail'], df_train['time_frame'])],
                   dtype=[('event', '?'), ('time', '<f8')])

# Compute Brier Score at specific times
brier_scores = brier_score(y_train, y_test, preds, times)
print(f"Brier Scores: {brier_scores}")

# Compute the Integrated Brier Score (IBS) over the specified time points
ibs = integrated_brier_score(y_train, y_test, preds, times)
print(f"Integrated Brier Score (IBS): {ibs}")

In [42]:
import numpy as np
from sksurv.metrics import cumulative_dynamic_auc
from sksurv.util import Surv
import pandas as pd

# Convert y_train and y_test to DataFrames
y_train = pd.DataFrame(y_train, columns=["GRF_STAT_PA_didFail", "time_frame"])
y_test = pd.DataFrame(y_test, columns=["GRF_STAT_PA_didFail", "time_frame"])

y_train["GRF_STAT_PA_didFail"] = y_train["GRF_STAT_PA_didFail"].astype("bool")
y_test["GRF_STAT_PA_didFail"] = y_test["GRF_STAT_PA_didFail"].astype("bool")

# Prepare survival objects for train and test data
y_test_surv = Surv.from_dataframe("GRF_STAT_PA_didFail", "time_frame", data=y_test)
y_train_surv = Surv.from_dataframe("GRF_STAT_PA_didFail", "time_frame", data=y_train)

# Risk scores for the test set
cum_hazards = rsf.predict_cumulative_hazard_function(X_test)
risk_scores = [np.mean(hazard.y) for hazard in cum_hazards]


# Define time points where you want to compute AUC (in days)
time_points = [0, 30, 365, 365*2, 365*3, 365*5, 356*10, max(y_test["time_frame"])-1]


# Calculate time-dependent AUC
auc_values, mean_auc = cumulative_dynamic_auc(survival_train=y_train_surv,
                                                 survival_test=y_test_surv,
                                                 estimate=risk_scores,
                                                 times=time_points)

# Print time-dependent AUC values
print("AUC at different time points:", auc_values)

# Calculate iAUC (mean AUC across time points using trapezoidal rule)
iAUC = np.trapz(auc_values, time_points) / (time_points[-1] - time_points[0])

print(f"Integrated AUC (iAUC): {iAUC}")


AUC at different time points: [0.55232329 0.6026965  0.62433201 0.64100273 0.64473615 0.64865954
 0.6213857  0.34107097]
Integrated AUC (iAUC): 0.5329915614696878
